# Upwork Jobs — Load & Explore

Use either a **clean CSV** produced by your parser or a **saved HTML** from Upwork search results.

**Instructions**:
1. Set `CSV_PATH` if you already have `upwork_clean.csv` (columns like `job_id, title, ...`).
2. Or set `HTML_PATH` to a saved Upwork HTML page and run the parser cells to generate the DataFrame.
3. Run the EDA cells to inspect rows, skills, price ranges, and simple charts.


In [ ]:
from pathlib import Path
import pandas as pd
import re

# ---- Configure one or both paths ----
# If you already have a CSV from the parser, point to it here:
CSV_PATH = Path('upwork_clean.csv')  # change to your CSV path

# If you saved raw Upwork HTML and want to parse inside this notebook, set this:
HTML_PATH = Path('test.html')       # change to your HTML path or leave None

pd.set_option('display.max_colwidth', 180)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 160)
print('Paths configured:', {'csv': str(CSV_PATH), 'html': str(HTML_PATH)})

## Optional: Parser for Upwork HTML
If `selectolax` is not installed, this cell will install it. Then it defines parsing helpers to read a saved Upwork search HTML and produce a DataFrame identical to the CSV schema.

In [ ]:
import sys
try:
    from selectolax.parser import HTMLParser
except Exception:
    # When running locally, this will install. It won't run in environments without internet access.
    %pip install selectolax --quiet
    from selectolax.parser import HTMLParser

from urllib.parse import urljoin
from datetime import datetime

BASE = 'https://www.upwork.com'
MONEY = re.compile(r"([$\u20ac\u00a3])\s*([\d.,]+)(?:\s*[-–]\s*([$\u20ac\u00a3])?\s*([\d.,]+))?")
JOB_ID = re.compile(r"_~([0-9a-z]+)", re.I)
HIGHLIGHT_NOISE = re.compile(r"\bspan[-\s]?class[-\s]?highlight\b|</?span[^>]*>", re.I)
WS = re.compile(r"\s+")

def clean_text(s: str) -> str:
    if not s:
        return ''
    s = HIGHLIGHT_NOISE.sub(' ', s)
    s = WS.sub(' ', s).strip()
    return s

def currency_code(sym: str) -> str:
    return {'$':'USD', '€':'EUR', '£':'GBP'}.get(sym, '')

def parse_price(article):
    price_type = ''
    price_min = price_max = ''
    currency = ''

    label = article.css_first("li[data-test='job-type-label'] strong")
    if label:
        t = label.text(strip=True).lower()
        if 'hour' in t:
            price_type = 'hourly'
        if 'fixed' in t or 'budget' in t:
            price_type = 'fixed'

    li = article.css_first("li[data-test='is-fixed-price'], li[data-test='is-hourly']")
    txt = li.text(separator=' ', strip=True) if li else article.text(separator=' ', strip=True)
    m = MONEY.search(txt)
    if m:
        sym1, a1, sym2, a2 = m.groups()
        currency = currency_code(sym1 or sym2 or '')
        def norm(x):
            return float(x.replace(',', ''))
        if a1 and a2:
            price_min, price_max = str(norm(a1)), str(norm(a2))
        elif a1:
            price_min = price_max = str(norm(a1))

    return price_type, currency, price_min, price_max

def parse_posted(article):
    time_el = article.css_first("time[datetime]")
    posted_at = time_el.attributes.get('datetime') if time_el else ''
    txt_all = clean_text(article.text(separator=' ', strip=True))
    m = re.search(r"posted\s+[^|]+ago", txt_all, re.I)
    posted_text = m.group(0) if m else ''
    return posted_at, posted_text

FRAMEWORK_DICT = [
    'Python','Scrapy','Selenium','BeautifulSoup','Requests','Pandas','NumPy',
    'Power BI','Tableau','Google Maps API','LinkedIn','GitHub','JavaScript','Node.js',
    'Playwright','Excel','Regex','API','ETL','Airflow'
]

def extract_frameworks(text_blob, skills_list):
    blob = f"{text_blob} {' '.join(skills_list)}".lower()
    found, seen, out = [], set(), []
    for fw in FRAMEWORK_DICT:
        if fw.lower() in blob:
            found.append(fw)
    for x in found:
        if x not in seen:
            out.append(x); seen.add(x)
    return ', '.join(out)

def parse_article(a):
    a_title = a.css_first("a[data-test='job-tile-title-link']")
    title = clean_text(a_title.text(strip=True) if a_title else '')
    url = urljoin(BASE, a_title.attributes.get('href', '')) if a_title else ''
    url = url.split('?', 1)[0]
    job_id = ''
    if url:
        m = JOB_ID.search(url)
        if m:
            job_id = m.group(1)

    p = a.css_first('p.text-body-sm') or a.css_first('.air3-line-clamp p, .air3-line-clamp')
    desc = clean_text(p.text(separator=' ', strip=True) if p else '')

    skills = [clean_text(s.text(strip=True)) for s in a.css("div.air3-token-container button.air3-token span") if clean_text(s.text(strip=True))]

    price_type, currency, price_min, price_max = parse_price(a)
    level_el = a.css_first("li[data-test='experience-level'] strong")
    level = level_el.text(strip=True) if level_el else ''
    posted_at, posted_text = parse_posted(a)

    # naive role inference
    t = f"{title} {desc}".lower()
    s = ' '.join(skills).lower()
    role = ''
    if 'power bi' in t or 'power bi' in s:
        role = 'Power BI Developer'
    elif 'data analyst' in t or 'data analysis' in t:
        role = 'Data Analyst'
    elif 'web scraping' in t or 'scraping' in t:
        role = 'Web Scraping Specialist'
    elif 'python' in t or 'python' in s:
        role = 'Python Developer'
    elif 'etl' in t or 'etl' in s:
        role = 'ETL Engineer'

    frameworks = extract_frameworks(f"{title} {desc}", skills)

    return {
        'platform': 'upwork',
        'job_id': job_id,
        'title': title,
        'description': desc,
        'role': role,
        'level': level,
        'skills': ', '.join(skills),
        'frameworks': frameworks,
        'price_type': price_type,
        'currency': currency,
        'price_min': price_min,
        'price_max': price_max,
        'posted_at': posted_at,
        'posted_text': posted_text,
        'url': url,
    }

def parse_upwork_html_to_df(html_path: Path) -> pd.DataFrame:
    html = html_path.read_text(encoding='utf-8', errors='ignore')
    dom = HTMLParser(html)
    arts = dom.css('article.job-tile')
    rows = [parse_article(a) for a in arts]
    return pd.DataFrame(rows)


## Load data
Choose one of the two blocks below: **load CSV** or **parse HTML**.

In [ ]:
df = None
if CSV_PATH and CSV_PATH.exists():
    df = pd.read_csv(CSV_PATH)
    print(f"Loaded CSV with {len(df)} rows from {CSV_PATH}")
elif HTML_PATH and HTML_PATH.exists():
    df = parse_upwork_html_to_df(HTML_PATH)
    print(f"Parsed HTML with {len(df)} rows from {HTML_PATH}")
else:
    raise FileNotFoundError('Neither CSV_PATH nor HTML_PATH exists. Set one of them to a valid file.')

df.head()

## Basic overview

In [ ]:
print('Shape:', df.shape)
print('Columns:', list(df.columns))
display_cols = [c for c in ['platform','job_id','title','role','level','price_type','currency','price_min','price_max','posted_at','url'] if c in df.columns]
df[display_cols].head(20)

## Skills analysis

In [ ]:
def split_skills(s):
    if pd.isna(s):
        return []
    return [x.strip() for x in str(s).split(',') if x.strip()]

skills_series = df.get('skills')
if skills_series is not None:
    exploded = df.assign(skill=df['skills'].fillna('').apply(split_skills)).explode('skill')
    top_skills = exploded[exploded['skill']!='']['skill'].value_counts().head(20)
    print(top_skills)
else:
    print('No skills column present.')

## Plot: Top 15 skills

In [ ]:
import matplotlib.pyplot as plt

skills_series = df.get('skills')
if skills_series is not None:
    exploded = df.assign(skill=df['skills'].fillna('').apply(split_skills)).explode('skill')
    vc = exploded[exploded['skill']!='']['skill'].value_counts().head(15)
    plt.figure()
    vc.sort_values().plot(kind='barh')
    plt.title('Top 15 Skills')
    plt.xlabel('Count')
    plt.ylabel('Skill')
    plt.tight_layout()
    plt.show()
else:
    print('No skills column present for plotting.')

## Price summary

In [ ]:
cols = [c for c in ['price_type','currency','price_min','price_max'] if c in df.columns]
if cols:
    display(df[cols].head())
    if 'price_min' in df.columns:
        with pd.option_context('display.float_format', '{:,.2f}'.format):
            print('price_min describe:\n', pd.to_numeric(df['price_min'], errors='coerce').dropna().describe())
    if 'price_max' in df.columns:
        with pd.option_context('display.float_format', '{:,.2f}'.format):
            print('price_max describe:\n', pd.to_numeric(df['price_max'], errors='coerce').dropna().describe())
else:
    print('No price columns present.')